# Nearest Neighbors (kNN)


### Objective

This notebook aims use nearest neighbors (kNN) model in the classification problem. In this case, we will use the Endometrium vs. Uterus cancer dataset and the goal is to separate **endometrium** tumors from the **uterine** ones.


### Learning objective

After finished this notebook, you should be able to explain **nearest neighbors (kNN) model** and its implementation by scikit-learn implementation


### Requirements

1. This notebook was created using
  * python 3.7.1
  * numpy 1.15.4
  * matplotlib 3.0.2
  * scikit-learn 0.20.1

2. You can check your Python version by running
```python
import sys
print(sys.version)
```

3. and the version of any module by running
```python
import <module name>
print(<module name>.__version__)
```

We will focus successively on decision trees, bagging trees and random forests.

the to introduce tree-based methods on classification problems.


We will implement our own nearest neighbors classifier and compare the results with the in-built classifiers offered by scikit-learn. 

Let us start by setting up our environment, loading the data, and setting up our cross-validation.

In [ ]:
import numpy as np
import pandas as pd

%pylab inline

Load the https://esmeml.github.io/data/endometrium_uterus_tumor.csv dataset

In [ ]:
from sklearn import preprocessing

endometrium_data = pd.read_csv('https://esmeml.github.io/data/endometrium_uterus_tumor.csv', sep=",")
endometrium_data.head(n=5)

X = endometrium_data.drop(['ID_REF', 'Tissue'], axis=1).values
y = pd.get_dummies(endometrium_data['Tissue']).values[:,1]

print(X.shape)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
# %load crossvalidation.py
def cross_validate_clf(design_matrix, labels, classifier, cv_folds):
    """ Perform a cross-validation and returns the predictions.
    
    Parameters:
    -----------
    design_matrix: (n_samples, n_features) np.array
        Design matrix for the experiment.
    labels: (n_samples, ) np.array
        Vector of labels.
    classifier:  sklearn classifier object
        Classifier instance; must have the following methods:
        - fit(X, y) to train the classifier on the data X, y
        - predict_proba(X) to apply the trained classifier to the data X and return probability estimates 
    cv_folds: sklearn cross-validation object
        Cross-validation iterator.
        
    Return:
    -------
    pred: (n_samples, ) np.array
        Vectors of predictions (same order as labels).
    """
    pred = np.zeros(labels.shape)
    for tr, te in cv_folds:
        classifier.fit(design_matrix[tr,:], labels[tr])
        pos_idx = list(classifier.classes_).index(1)
        pred[te] = (classifier.predict_proba(design_matrix[te,:]))[:, pos_idx]
    return pred

def cross_validate_clf_optimize(design_matrix, labels, classifier, cv_folds):
    """ Perform a cross-validation and returns the predictions.
    
    Parameters:
    -----------
    design_matrix: (n_samples, n_features) np.array
        Design matrix for the experiment.
    labels: (n_samples, ) np.array
        Vector of labels.
    classifier:  sklearn classifier object
        Classifier instance; must have the following methods:
        - fit(X, y) to train the classifier on the data X, y
        - predict_proba(X) to apply the trained classifier to the data X and return probability estimates 
    cv_folds: sklearn cross-validation object
        Cross-validation iterator.
        
    Return:
    -------
    pred: (n_samples, ) np.array
        Vectors of predictions (same order as labels).
    """
    pred = np.zeros(labels.shape)
    for tr, te in cv_folds:
        classifier.fit(design_matrix[tr,:], labels[tr])
        print(classifier.best_params_)
        pos_idx = list(classifier.best_estimator_.classes_).index(1)
        pred[te] = (classifier.predict_proba(design_matrix[te,:]))[:, pos_idx]
    return pred

In [ ]:
from sklearn import model_selection

def stratifiedMFolds(y, num_folds):
    kf = model_selection.StratifiedKFold(n_splits=num_folds)
    folds_regr = [(tr, te) for (tr, te) in kf.split(np.zeros(y.size), y)]
    return folds_regr

Now create 10 cross validate folds on the data. 

In [ ]:
cv_folds = stratifiedMFolds(y, 10)

Import the previously written cross validation function.

In [ ]:
# let's redefine the cross-validation procedure with standardization
from sklearn import preprocessing
def cross_validate(design_matrix, labels, regressor, cv_folds):
    """ Perform a cross-validation and returns the predictions. 
    Use a scaler to scale the features to mean 0, standard deviation 1.
    
    Parameters:
    -----------
    design_matrix: (n_samples, n_features) np.array
        Design matrix for the experiment.
    labels: (n_samples, ) np.array
        Vector of labels.
    classifier:  Regressor instance; must have the following methods:
        - fit(X, y) to train the regressor on the data X, y
        - predict_proba(X) to apply the trained regressor to the data X and return predicted values
    cv_folds: sklearn cross-validation object
        Cross-validation iterator.
        
    Return:
    -------
    pred: (n_samples, ) np.array
        Vectors of predictions (same order as labels).
    """
    
    n_classes = np.unique(labels).size
    pred = np.zeros((labels.shape[0], n_classes))
    for tr, te in cv_folds:
        scaler = preprocessing.StandardScaler()
        Xtr = scaler.fit_transform(design_matrix[tr,:])
        ytr =  labels[tr]
        Xte = scaler.transform(design_matrix[te,:])
        regressor.fit(Xtr, ytr)
        pred[te, :] = regressor.predict_proba(Xte)
    return pred

# 1. *k*-Nearest Neighbours Classifier

A k-neighbours classifier can be initialised as `knn_clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k)`

Cross validate 20 *k*-NN classifiers on the loaded datset using `cross_validate`. 

In [ ]:
from sklearn import neighbors
from sklearn import metrics

aurocs_clf = []
# Create a range of values of k. We will use this throughout this notebook.
k_range    = range(1,40,2) 

for k in k_range:
    clf_knn    = # TODO
    y_pred = cross_validate(X, y, clf_knn, cv_folds)
    
    fpr, tpr, thresholdss = metrics.roc_curve(y, y_pred[:,1])
    aurocs_clf.append(metrics.auc(fpr,tpr))

Now plot the AUC as a function of the number of nearest neighbours chosen.

In [ ]:
plt.plot(k_range, aurocs_clf, color='blue')
plt.xlabel('Number of nearest neighbours')
plt.ylabel('Cross-validated AUC')
plt.title('Nearest neighbours classification - cross validated AUC.')

**Question.** Find the best value for the parameter `n_neighbors` by finding the one that gives the maximum value of AUC.

In [ ]:
best_k = np.argmax(aurocs_clf)
print(k_range[best_k])

Let us now use `sklearn.model_selection.GridSearchCV` do to the same. The parameter to be cross-validated is the number of nearest neighbours to choose. Use an appropriate list to feed to `GridSearchCV` to find the best value for the nearest neighbours parameter.

In [ ]:
from sklearn import model_selection
from sklearn import metrics

classifier = neighbors.KNeighborsClassifier()

param_grid = {'n_neighbors': k_range}
clf_knn_opt = #TODO
clf_knn_opt.fit(X, y)

#### Finding the best parameter

In [ ]:
print (clf_knn_opt.best_params_)

Even when we use the same scoring strategy, the optimum is different due to the cross-validation strategy implemented by scikit-learn, which computs one AUC per fold and averages them.

Try choosing different scoring metrics for GridSearchCV, and see how the result changes. You can find scoring metrics [here](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

Now compare the performance of the *k*-nearest neighbours classifier with logistic regularisation (both, non-regularised, and regularised).

In [ ]:
from sklearn import linear_model

clf_logreg_l2 = linear_model.LogisticRegression()
logreg_params = {'C': np.logspace(-3, 3, 7), 
                 'penalty': ['l2'],
                 'solver': ['liblinear']}
                
clf_logreg_opt = model_selection.GridSearchCV(clf_logreg_l2, 
                                              logreg_params, 
                                              scoring='roc_auc',
                                              cv=3,
                                              iid=True
                                             )
clf_logreg_opt.fit(X, y)
ypred_clf_logreg_opt = cross_validate(X, y, clf_logreg_opt.best_estimator_, cv_folds)
fpr_clf_logreg_opt, tpr_clf_logreg_opt, thresh = metrics.roc_curve(y, ypred_clf_logreg_opt[:,1])

In [ ]:
clf_logreg = linear_model.LogisticRegression(C=1e12,solver='liblinear')
ypred_clf_logreg = cross_validate(X, y, clf_logreg, cv_folds)
fpr_clf_logreg, tpr_clf_logreg, thresh = metrics.roc_curve(y, ypred_clf_logreg[:,1], pos_label=1)

In [ ]:
ypred_clf_knn_opt = # TODO
fpr_clf_knn_opt, tpr_clf_knn_opt, thresh =  #TODO

In [ ]:
logreg_l2_h,  = plt.plot(fpr_clf_logreg_opt, tpr_clf_logreg_opt, 'b-')
logreg_h,     = plt.plot(fpr_clf_logreg, tpr_clf_logreg, 'g-')
knn_h,        = plt.plot(fpr_clf_knn_opt, tpr_clf_knn_opt, 'r-')
logreg_l2_auc = metrics.auc(fpr_clf_logreg_opt, tpr_clf_logreg_opt)
logreg_auc    = metrics.auc(fpr_clf_logreg, tpr_clf_logreg)
knn_auc       = metrics.auc(fpr_clf_knn_opt, tpr_clf_knn_opt)


logreg_legend    = 'LogisticRegression. AUC=%.2f' %(logreg_auc)
logreg_l2_legend = 'Regularised LogisticRegression. AUC=%.2f' %(logreg_l2_auc)
knn_legend       = 'KNeighborsClassifier. AUC=%.2f' %(knn_auc)
plt.legend([logreg_h, logreg_l2_h, knn_h], [logreg_legend, logreg_l2_legend, knn_legend])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves comparison for logistic regression and k-nearest neighbours classifier.')
plt.show()

kNN performs worst than the logistic regression, which is expected due to the large number of features

**Setting the distance measure**. You will notice that *k*-nearest neighbours classifiers measure distances between points to determine similarity. By default, we use the Euclidean distance metric. Often, using other distance metrics can prove to be helpful. Try to change the distance metric used here by passing it as an argument to the declaration of the classifier. 

In [ ]:
classifiers = {}            
y_preds     = {}

# Fix a set of distance metrics to use
d_metrics = ['euclidean', 'cityblock', 'correlation' , 'cosine']
aurocs    = {}      

for m in d_metrics:
    aurocs[m] = []          
    for k in k_range: 
        classifiers[m] = neighbors.KNeighborsClassifier(n_neighbors=k, metric=m) # TODO. Initialise a kNN classifier with n_neighbors=k and metric=m
        y_preds[m]     = cross_validate(X, y, classifiers[m], cv_folds)# TODO. Cross validate on the data. Use cross_validate
    
        fpr, tpr, thresholds = metrics.roc_curve(y, y_preds[m][:,1])# TODO.
        auc                  = metrics.auc(fpr, tpr)# TODO.
        aurocs[m].append(auc)     
        
        print('Metric = %-12s | k = %3d | AUC = %.3f.' %(m, k, aurocs[m][-1]))

Now plot ROC curves for all the metrics together.

In [ ]:
f = plt.figure()
handles = [[] for i in range(len(d_metrics))]
colors = ['blue', 'red', 'black', 'yellow'] 

for i in range(len(d_metrics)):              
    handles[i], = plt.plot(k_range, aurocs[d_metrics[i]], color=colors[i])

plt.xlabel('Number of nearest neighbors', fontsize=16)
plt.ylabel('Cross-validated AUC', fontsize=16)
plt.title('Nearest neighbors classification', fontsize=16)
plt.legend(handles, d_metrics)